In [1]:
import numpy as np
from astropy import units as u 
from astropy.io import fits
import matplotlib.pyplot as plt
import aplpy
from math import ceil
from astropy import stats 
from glob import glob
import matplotlib as mpl
from tqdm import tqdm

import sys
sys.path.append('/Users/abarnes/GitHub/misc/')
from data_cube_analysis import getdata

plt.style.use('paper.mplstyle')

In [2]:
def get_channel_maps(input, start=-200, stop=200, step=20):
    """
    This function calculates the zeroth moment (total intensity) for each velocity channel in a data cube,
    and writes the results to a FITS file.

    Args:
    input (str, cube): Filename or cube object... 
    start (int): The start of the velocity range for channel extraction. Default is -200 km/s.
    stop (int): The end of the velocity range for channel extraction. Default is 200 km/s.
    step (int): The velocity step size for channel extraction. Default is 20 km/s.

    Returns:
    None
    """

    # Use the get_cube function from the getdata module to extract the data cube from the FITS file.
    cube = getdata.get_cube(input)
    # if isinstance(input, str):
    #     cube = getdata.get_cube(input)
    # else: 
    #     cube = input
    
    # Define the velocity channels.
    channels = np.arange(start, stop, step) *u.km/u.s

    # Get the number of channels.
    n = len(channels)-1

    # Initialize an array to hold the channel map data.
    channelmaps_data = ['']*(n)

    # For each channel...
    for i in tqdm(range(n)): 

        # ...extract the cube for that channel...
        cube_channels = cube.spectral_slab(channels[i], channels[i+1])

        # ...and calculate the zeroth moment, storing the result in the array.
        channelmaps_data[i] = cube_channels.moment0().hdu.data
        channelmaps_data[i][channelmaps_data[i] == 0] = np.nan

    # Get the FITS header from the last cube's zeroth moment.
    channelmaps_header = cube_channels.moment0().hdu.header

    # Create a FITS HDU using the channel map data and the header.
    channelmaps_hdu = fits.PrimaryHDU(channelmaps_data, channelmaps_header)

    # Define the output file name by removing '.fits' from the input file name.
    outputfile = inputfile.replace('.fits', '')
 
    # Write the HDU to a FITS file.
    channelmaps_hdu.writeto('%s_chans.fits' %outputfile, overwrite=True)

    return(channelmaps_hdu)

In [3]:
def plot_channel_maps_indv(channelmaps_hdu, fileappend='', outputdir='../figures/', start=-200, stop=200, step=20, vmin=None, vmax=None, std=None, l=0.8065474, b=-0.1999737, width=0.2*0.9, height=0.2*0.9):
    """
    This function plots all the channel maps contained in a FITS HDU in a grid with a specified number of columns.
    It shows the velocity range in the upper right corner of each plot.

    Args:
    channelmaps_hdu (HDU): The FITS HDU containing the channel maps.
    start (int): The start of the velocity range for channel extraction. Default is -200 km/s.
    stop (int): The end of the velocity range for channel extraction. Default is 200 km/s.
    step (int): The velocity step size for channel extraction. Default is 20 km/s.

    Returns:
    None
    """
    # Determine the number of channels from the FITS HDU.
    channels = channelmaps_hdu.header['NAXIS3']
    
    # For each channel...
    for chan in range(channels):
        
        # if chan!=0:
        #     continue

        # ...create a copy of the FITS HDU...
        channelmaps_hdu_ = channelmaps_hdu.copy()
        
        # ...replace the data in the HDU with the data for the current channel...
        channelmaps_hdu_.data = channelmaps_hdu_.data[chan]
        
        # Initialize the plot figure.
        # ...create a subplot in the appropriate position in the grid...
        fig = plt.figure(figsize=(5,5))
        gc = aplpy.FITSFigure(channelmaps_hdu_, subplot=(1, 1, 1), figure=fig)  # 4 columns
        gc.recenter(l, b, width=width, height=height)
        
        # ...display the channel map in grayscale and with contours...
        # gc.show_grayscale()
        gc.show_colorscale(cmap='inferno', vmin=vmin, vmax=vmax)
        # gc.show_contour(channelmaps_hdu_, levels=5)
        
        if std is not None: 
            gc.show_contour(channelmaps_hdu_, levels=[3*std], linewidths=0.9, colors='white', smooth=3)

        # Remove ticks etc
        gc.axis_labels.hide()
        gc.tick_labels.hide()
        gc.ticks.set_color('black')
        gc.ticks.set_linewidth(2)
        gc.set_nan_color('lightgrey')

        # ...and display the velocity range in the upper right corner.
        velo_range = f"{start + chan*step} to {start + (chan+1)*step} km/s"
        gc.add_label(0.05, 0.95, velo_range, relative=True, color='black', bbox=dict(facecolor='white', boxstyle='round', alpha=0.75), ha='left')
            
        gc.show_ellipses(0.8119, -0.1868, 240/3600, 240/3600, edgecolor='white', linestyles='dotted', linewidths=2)
    
        # Adjust the subplot layout to reduce space between subplots
        fig.tight_layout(h_pad=0, w_pad=-0.6)
        plt.subplots_adjust(wspace=-0.06, hspace=0)
        
        # Save the figure. 
        fig.savefig(outputdir+fileappend+'_%i.png' %chan, bbox_inches='tight')
    
        plt.close('all')

    return()

For larger region, using cube from Adam.... 

In [4]:
inputfile = '/Users/abarnes/Dropbox/work/Projects/MOPRA/CMZ/3mm/CMZ_3mm_HNCO.fits'
print(inputfile)
hdu = fits.open(inputfile)[0]
print(hdu)

/Users/abarnes/Dropbox/work/Projects/MOPRA/CMZ/3mm/CMZ_3mm_HNCO.fits


In [5]:
cube = getdata.get_cube(hdu)
ds9_str = 'galactic; box(0.6994511,-0.0888193,1822.851",1485.249",0.13374212)'
subcube = cube.subcube_from_ds9region(ds9_str) 
hdu = subcube.hdu

        Use `regions.Regions.parse` instead. [spectral_cube.spectral_cube]


In [6]:
channelmaps_hdu = get_channel_maps(hdu, 
                                   start=-40, 
                                   stop=130, 
                                   step=2)

100%|██████████| 84/84 [00:08<00:00,  9.53it/s]


In [7]:
# ensuring plotting over same scale
data = channelmaps_hdu.data.copy()
vmin, vmax = np.nanpercentile(data, [0, 99.5])
std = stats.mad_std(data, ignore_nan=True)
std = stats.mad_std(data[data<3*std], ignore_nan=True)
print(vmin, vmax, std)

plot_channel_maps_indv(channelmaps_hdu, 
                  start = -40, 
                  stop = 130, 
                  step = 2, 
                  vmin = vmin, 
                  vmax = vmax, 
                  std = None,
                  l=0.6994511, 
                  b=-0.0888193, 
                  width=0.5063474, 
                  height=0.4125693,
                  fileappend = 'hnco_chans_large_mopra/hcno_chans')

-0.9643024206161499 6.5068066763877885 0.18577231894232782


()

In [8]:
import imageio
filenames = glob('../figures/hnco_chans_large_mopra/*.png')
filenames_sorted = sorted(filenames, key=lambda x: int(x.split('_')[-1].split('.')[0]))
images = []
for filename in filenames_sorted:
    images.append(imageio.imread(filename))
imageio.mimsave('../figures/hnco_chans_large_mopra/hnco_chans_large_mopra_movie.gif', images)

/var/folders/3y/cmsgt3_x7x72bbs__zhy_q6h0008wr/T/ipykernel_81180/2821716098.py:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(filename))
